In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LassoCV

import warnings
warnings.filterwarnings("ignore")

# from math import sqrt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [5]:
###############################################################################
### local imports                                                           ###
###############################################################################

import acquire as acq
import prepare as prep

from debug import local_settings, timeifdebug, timeargsifdebug, frame_splain
from dfo import DFO, set_dfo

In [6]:
from wrangle_zillow import get_zillow_data, prep_zillow_data, sqls

Acquire & Summarize
Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid .
Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
Only include properties that include a latitude and longitude value.

In [8]:
dfo = set_dfo(get_zillow_data(sql=sqls['mvp']), splain=True)

In [10]:
df = dfo.df

In [11]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,logerror
0,14297519,3.5,4.0,3100.0,0.025595
1,17052889,1.0,2.0,1465.0,0.055619
2,14186244,2.0,3.0,1243.0,0.005383
3,12177905,3.0,4.0,2376.0,-0.103410
4,12095076,3.0,4.0,2962.0,-0.001011


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
y = df['logerror']
X= df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [20]:
print(train.shape); print(test.shape)

(41728, 5)
(10433, 5)


#### Training the Model
Now its time to train our model on our training data!

In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
lm = LinearRegression()

In [23]:
lm.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [31]:
lm.fit(X_train, y_train)
print("Linear Model:", lm)

lm_y_intercept = lm.intercept_
print("intercept: ", lm_y_intercept)

lm_coefficients = lm.coef_
print("coefficients: ", lm_coefficients)

Linear Model: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
intercept:  -0.004516590056757357
coefficients:  [-1.52193288e-03  4.82509029e-03  5.48131807e-06]


In [35]:
print('{} = b + m1 * {} + m2 * {}'.format(y_train, X_train,X_train))
print('    y-intercept  (b): %.2f' % lm_y_intercept)
print('    coefficient (m1): %.2f' % lm_coefficients[0][0])
print('    coefficient (m2): %.2f' % lm_coefficients[0][1])

14357   -0.009250
25617   -0.033362
46743    0.079581
28702    0.036173
31222    0.164710
17139    0.008093
17334    0.524423
7250     0.001454
18837   -0.013678
28573    0.081074
43354   -0.003688
14996    0.116986
14863   -0.052676
39271    0.435081
3554    -0.094097
47375   -0.019032
32338   -0.008857
22340    0.044632
48939    0.073533
19221    0.097824
23502    0.032605
17558    0.153572
7823    -0.049993
37957   -0.005219
27741   -0.936589
13267    0.226321
32669   -0.053511
23214   -0.016213
23418   -0.020084
45928    0.678121
           ...   
22461    0.002747
6648     0.117729
30255    0.038662
52158    0.013209
49191    0.008621
6484     0.024015
26626    0.081715
45507    0.042394
50999    0.060059
7985     0.050269
17476   -0.011551
14944    0.044140
31140   -0.009510
3325     0.107358
35662    0.048804
17747    0.018510
36249    0.003024
111     -0.186837
33710    0.013369
5664     0.047396
23166    0.013743
96       0.164707
22241    0.013504
23766    0.001934
46203    0

IndexError: invalid index to scalar variable.

In [36]:
y_pred_lm = lm.predict(X_train)

In [40]:
mse_lm = mean_squared_error(y_train, y_pred_lm)
print("linear model\n  mean squared error: {:.3}".format(mse_lm)) 

r2_lm = r2_score(y_train, y_pred_lm)
print((r2_lm))

linear model
  mean squared error: 0.0312
0.001934026314660997


In [41]:
from math import sqrt

y_pred_baseline = np.array([y_train.mean()[0]]*len(y_train))
MSE = mean_squared_error(y_train, y_pred_baseline)
SSE = MSE*len(y_train)
RMSE = sqrt(MSE)

evs = explained_variance_score(y_train, y_pred_baseline)

TypeError: 'float' object is not subscriptable